For the Online Retail dataset, typical gold layer tables include aggregated business metrics like:

Revenue by Country / Month / Product
Top Customers (by spend)
Customer RFM (Recency, Frequency, Monetary) scoring
Product performance (best sellers, returns)

In [0]:
# ---- Parameters (CI/CD compatible) ----
dbutils.widgets.text("storage_account", "retaildatalaketest")
dbutils.widgets.text("container", "ecommerce")

storage_account = dbutils.widgets.get("storage_account")
container = dbutils.widgets.get("container")

base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net"

In [0]:
from pyspark.sql import functions as F

# silver_path = "abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/silver/online_retail_silver"
# gold_path = "abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/gold/"

silver_path = f"{base_path}/silver/online_retail_silver"
gold_path = f"{base_path}/gold"

# Read silver
silver_df = spark.read.format("delta").load(silver_path)

assert silver_df.count() > 0, "❌ Silver layer is empty"

# Step 1: Extract just the date part as string, then cast to DATE (simple & safe)
silver_clean = silver_df.withColumn(
    "InvoiceDate",
    F.to_date(F.col("InvoiceDate"), "M/d/yy H:mm")   # Handles 12/1/10 8:26 → 2010-12-01
)
silver_clean.limit(5).display()
# Step 2: Add TotalAmount
silver_with_amount = silver_clean.withColumn(
    "TotalAmount", 
    F.col("Quantity") * F.col("Price")
)

# --- Gold Metrics (Simple Aggregations) ---

# 1. Monthly Revenue by Country
monthly_revenue = (silver_with_amount
    .groupBy(
        F.date_format("InvoiceDate", "yyyy-MM").alias("YearMonth"),
        "Country"
    )
    .agg(F.round(F.sum("TotalAmount"), 2).alias("Revenue"))
    .orderBy("YearMonth", F.desc("Revenue"))
)

# 2. Top 20 Products by Revenue
top_products = (silver_with_amount
    .groupBy("StockCode", "Description")
    .agg(
        F.round(F.sum("TotalAmount"), 2).alias("TotalRevenue"),
        F.sum("Quantity").alias("TotalSold")
    )
    .orderBy(F.desc("TotalRevenue"))
    .limit(20)
)

# 3. Top 10 Customers by Spend
top_customers = (silver_with_amount
    .groupBy("CustomerID")
    .agg(
        F.round(F.sum("TotalAmount"), 2).alias("TotalSpend"),
        F.countDistinct("Invoice").alias("NumOrders")
    )
    .orderBy(F.desc("TotalSpend"))
    .limit(10)
)

# 4. Daily Summary (useful for dashboards)
# 4. Daily Summary - Now with year/month as real columns + partitioning
daily_summary = (silver_clean
    .withColumn("InvoiceDate", F.to_date("InvoiceDate"))  # Clean date
    .withColumn("year", F.year("InvoiceDate"))
    .withColumn("month", F.month("InvoiceDate"))
    .groupBy("year", "month", "InvoiceDate")
    .agg(
        F.countDistinct("Invoice").alias("NumOrders"),
        F.round(F.sum("TotalAmount"), 2).alias("DailyRevenue"),
        F.countDistinct("CustomerID").alias("UniqueCustomers")
    )
    .orderBy("InvoiceDate")
    .select("InvoiceDate", "year", "month", "NumOrders", "DailyRevenue", "UniqueCustomers")  # Explicit select
)


# Save as separate simple Delta tables in gold
monthly_revenue.write.format("delta").mode("overwrite").save(gold_path + "monthly_revenue")
top_products.write.format("delta").mode("overwrite").save(gold_path + "top_products")
top_customers.write.format("delta").mode("overwrite").save(gold_path + "top_customers")
daily_summary.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .option("overwriteSchema", "true") \
    .save(f"{gold_path}/daily_summary")

assert monthly_revenue.count() > 0, "❌ monthly_revenue empty"
assert top_products.count() >= 10, "❌ insufficient top products"
assert top_customers.count() > 0, "❌ top customers empty"

# Preview
print("Monthly Revenue by Country (Top 10)")
monthly_revenue.limit(10).display()

print("Top Products")
top_products.display()

print("Top Customers")
top_customers.display()

print("Daily summary")
daily_summary.limit(5).display()


Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,UnitPrice,TotalAmount
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom,2.55,15.299999999999999
536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom,3.39,20.34
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom,2.75,22.0
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom,3.39,20.34
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom,3.39,20.34


Monthly Revenue by Country (Top 10)


YearMonth,Country,Revenue
2010-12,United Kingdom,498661.85
2010-12,Germany,15241.14
2010-12,France,9616.31
2010-12,EIRE,8813.88
2010-12,Netherlands,8784.48
2010-12,Japan,7705.07
2010-12,Sweden,3834.3
2010-12,Norway,3787.12
2010-12,Portugal,2439.97
2010-12,Spain,1843.73


Top Products


StockCode,Description,TotalRevenue,TotalSold
23843,"PAPER CRAFT , LITTLE BIRDIE",168469.6,80995
22423,REGENCY CAKESTAND 3 TIER,142592.95,12402
85123A,WHITE HANGING HEART T-LIGHT HOLDER,100448.15,36725
85099B,JUMBO BAG RED RETROSPOT,85220.78,46181
23166,MEDIUM CERAMIC TOP STORAGE JAR,81416.73,77916
POST,POSTAGE,77821.96,3121
47566,PARTY BUNTING,68844.33,15291
84879,ASSORTED COLOUR BIRD ORNAMENT,56580.34,35362
M,Manual,53779.93,7173
23084,RABBIT NIGHT LIGHT,51346.2,27202


Top Customers


CustomerID,TotalSpend,NumOrders
14646,280206.02,73
18102,259657.3,60
17450,194550.79,46
16446,168472.5,2
14911,143825.06,201
12415,124914.53,21
14156,117379.63,55
17511,91062.38,31
16029,81024.84,63
12346,77183.6,1


Daily summary


InvoiceDate,year,month,NumOrders,DailyRevenue,UniqueCustomers
2010-12-01,2010,12,121,46376.49,95
2010-12-02,2010,12,137,47316.53,99
2010-12-03,2010,12,57,23921.71,50
2010-12-05,2010,12,87,31771.6,75
2010-12-06,2010,12,94,31215.64,82


In [0]:
from pyspark.sql.functions import sum, col
silver_path = "abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/silver/online_retail_silver"
gold_path = "abfss://ecommerce@retaildatalaketest.dfs.core.windows.net/gold/"

# Read silver
silver_df = spark.read.format("delta").load(silver_path)
gold_df = (
    silver_df
    .withColumn("TotalAmount", col("TotalAmount"))
    .groupBy("Country")
    .agg(sum("TotalAmount").alias("TotalRevenue"))
)

gold_df.write.format("delta").mode("overwrite").saveAsTable("gold")

gold_df.limit(5).display()
display(spark.sql("DESCRIBE EXTENDED gold"))

Country,TotalRevenue
Switzerland,56443.95000000004
Poland,7334.649999999997
Iceland,4309.999999999997
Lebanon,1693.8800000000003
Italy,17483.240000000005


col_name,data_type,comment
Country,string,null
TotalRevenue,double,null
,,
# Delta Statistics Columns,,
Column Names,"Country, TotalRevenue",
Column Selection Method,first-32,
,,
# Detailed Table Information,,
Catalog,adb,
Database,default,
